In [2]:

# Añadir el directorio raíz al path para importar los módulos



In [1]:
import os
import sys
import json
import logging
from typing import List, Dict, Optional, Any, Tuple
from dotenv import load_dotenv
from datetime import datetime

# Asegurarse de que puede encontrar el módulo incubator
sys.path.append(os.path.dirname(os.path.abspath("../")))

# Importar componentes mejorados
from incubator.llm.antropic import AnthropicClient
from incubator.agents.agent import Agent
from incubator.api.dialog_api import DialogWorkflowAPI
from incubator.wf.enhanced_workflow import EnhancedWorkflowEngine
from incubator.wf.enhanced_dialognode import EnhancedDialogNode
from incubator.messages.message import Message

# Cargar variables de entorno (para la API_KEY de Claude)
load_dotenv()

# Configurar logging
logging.basicConfig(
    level=logging.INFO,
    format='[%(asctime)s] %(levelname)s - %(name)s: %(message)s',
    datefmt='%H:%M:%S'
)

def run_enhanced_dialog_example():
    """
    Ejecuta un ejemplo de diálogo usando los componentes mejorados
    para solucionar el problema de captura de resultados.
    """
    print("\n==================================================")
    print("EJEMPLO DE DIÁLOGO MEJORADO - SOLUCIÓN AL PROBLEMA DE CAPTURA")
    print("==================================================\n")
    
    # Solicitar dominio y prompt inicial
    domain = input("Introduce un dominio para el diálogo (por ejemplo, 'marketing', 'programación', etc.): ")
    if not domain:
        domain = "estrategias de marketing digital"
    
    prompt = input("Introduce un prompt inicial para el diálogo: ")
    if not prompt:
        prompt = f"Desarrolla una estrategia innovadora de {domain} para una startup tecnológica"
    
    # Crear API de diálogo mejorada
    dialog_api = DialogWorkflowAPI(
        save_results=True,
        results_dir="resultados_mejorados",
        verbose=True
    )
    
    # Ejecutar diálogo
    print(f"\nEjecutando diálogo mejorado en dominio '{domain}'...")
    results = dialog_api.run_dialog(
        initial_prompt=prompt,
        domain=domain,
        max_iterations=3
    )
    
    # Mostrar resultados disponibles
    if "error" not in results:
        print("\n===== RESULTADOS DISPONIBLES =====")
        for key in results.keys():
            print(f"- {key}")
        
        # Mostrar conclusión si está disponible
        if "conclusion" in results:
            print("\n===== CONCLUSIÓN =====")
            conclusion = results["conclusion"]
            # Limitar a primeras 300 caracteres para la salida
            preview = conclusion[:300] + "..." if len(conclusion) > 300 else conclusion
            print(preview)
        
        # Mostrar contenido de la conversación 
        if "conversation" in results:
            print("\n===== FRAGMENTO DE CONVERSACIÓN =====")
            conversation = results["conversation"]
            # Mostrar primeras 5 líneas (aprox.)
            lines = conversation.split("\n\n")[:5]
            preview = "\n\n".join(lines)
            print(preview)
            print("...\n[Conversación truncada]")
        
        print("\nProceso completado exitosamente.")
    else:
        print(f"\nERROR: {results['error']}")

def run_manual_workflow_example():
    """
    Crea y ejecuta un workflow manualmente para demostrar el control
    detallado sobre los componentes mejorados.
    """
    print("\n==================================================")
    print("EJEMPLO DE CREACIÓN MANUAL DE WORKFLOW - MAYOR CONTROL")
    print("==================================================\n")
    
    # Solicitar dominio
    domain = input("Introduce un dominio para el diálogo manual (por ejemplo, 'inversiones', 'salud', etc.): ")
    if not domain:
        domain = "inversiones financieras"
    
    prompt = input("Introduce un prompt inicial para el diálogo: ")
    if not prompt:
        prompt = f"Analiza las mejores estrategias de {domain} para el próximo año"
    
    # Obtener API key
    api_key = os.getenv("ANTHROPIC_API_KEY")
    if not api_key:
        print("ERROR: No se encontró ANTHROPIC_API_KEY en las variables de entorno")
        return
    
    # Cliente LLM
    llm_client = AnthropicClient(api_key=api_key)
    
    # Crear los agentes personalizados
    expert = Agent(
        name="Expert",
        description=f"Experto en {domain}",
        llm_client=llm_client,
        system_prompt=f"""
        Eres un experto reconocido en el campo de {domain}.
        
        Tu tarea es proporcionar análisis profundos y perspectivas basadas en evidencia
        sobre temas relacionados con {domain}. Debes:
        
        1. Ofrecer explicaciones claras y detalladas
        2. Citar tendencias actuales y factores relevantes
        3. Proporcionar ejemplos concretos cuando sea necesario
        4. Responder a las preguntas del Analista de manera completa
        
        Mantén un tono profesional y asegúrate de que tus recomendaciones estén bien fundamentadas.
        """
    )
    
    analyst = Agent(
        name="Analyst",
        description=f"Analista crítico de {domain}",
        llm_client=llm_client,
        system_prompt=f"""
        Eres un analista crítico especializado en evaluar propuestas y análisis de {domain}.
        
        Tu tarea es examinar críticamente la información proporcionada por el Experto,
        identificar posibles sesgos, lagunas o áreas de mejora. Debes:
        
        1. Hacer preguntas incisivas que exploren diferentes ángulos
        2. Señalar posibles debilidades o limitaciones en los argumentos
        3. Sugerir consideraciones adicionales relevantes
        4. En tu evaluación final, proporcionar un análisis FODA (Fortalezas, Oportunidades, 
           Debilidades, Amenazas) de las ideas presentadas
        
        Cuando hayas completado tu análisis, marca claramente tu conclusión con "CONCLUSIÓN ANALÍTICA:"
        seguida de un resumen estructurado de las principales conclusiones.
        """
    )
    
    # Configurar marcadores de contenido personalizados
    content_markers = {
        "conclusion": "CONCLUSIÓN ANALÍTICA:",
        "swot_analysis": "ANÁLISIS FODA:",
        "recommendations": "RECOMENDACIONES:"
    }
    
    # Crear nodo de diálogo mejorado
    dialog_node = EnhancedDialogNode(
        id="expert_analyst_dialog",
        agent_a=expert,
        agent_b=analyst,
        max_iterations=3,
        terminate_on_keywords=list(content_markers.values()),
        content_markers=content_markers,
        expected_ports=["default", "conversation", "conclusion", "swot_analysis", 
                        "recommendations", "expert_final", "analyst_final"],
        enable_auto_correction=True,
        debug_mode=True,
        verbose=True
    )
    
    # Crear workflow mejorado
    workflow = EnhancedWorkflowEngine(
        workflow_id=f"manual_workflow_{domain.replace(' ', '_')}",
        expected_node_outputs={
            "expert_analyst_dialog": ["default", "conversation", "conclusion", 
                                     "swot_analysis", "recommendations", 
                                     "expert_final", "analyst_final"]
        },
        enable_auto_correction=True,
        debug=True
    )
    
    # Añadir nodo al workflow
    workflow.add_node(dialog_node, is_input=True, is_output=True)
    
    # Preparar directorio de resultados
    results_dir = "resultados_workflow_manual"
    os.makedirs(results_dir, exist_ok=True)
    
    # Ejecutar workflow
    print(f"\nEjecutando workflow manual con diálogo sobre {domain}...")
    results = workflow.execute(
        inputs={"expert_analyst_dialog": prompt},
        save_results=True,
        results_dir=results_dir
    )
    
    # Mostrar resultados disponibles
    if "expert_analyst_dialog" in results:
        outputs = results["expert_analyst_dialog"]
        
        print("\n===== PUERTOS DE SALIDA DISPONIBLES =====")
        for port in outputs.keys():
            print(f"- {port}")
        
        # Mostrar análisis FODA si está disponible
        if "swot_analysis" in outputs:
            print("\n===== ANÁLISIS FODA =====")
            swot = outputs["swot_analysis"]["content"]
            # Limitar a primeras 300 caracteres para la salida
            preview = swot[:300] + "..." if len(swot) > 300 else swot
            print(preview)
        
        # Mostrar conclusión si está disponible
        if "conclusion" in outputs:
            print("\n===== CONCLUSIÓN ANALÍTICA =====")
            conclusion = outputs["conclusion"]["content"]
            # Limitar a primeras 300 caracteres para la salida
            preview = conclusion[:300] + "..." if len(conclusion) > 300 else conclusion
            print(preview)
        
        # Mostrar diagnóstico
        print("\n===== DIAGNÓSTICO DEL WORKFLOW =====")
        diagnóstico = workflow.monitor.diagnose_workflow_issues()
        
        if diagnóstico["issues"]:
            print("\nProblemas detectados:")
            for issue in diagnóstico["issues"]:
                print(f"- {issue}")
        
        if diagnóstico["warnings"]:
            print("\nAdvertencias:")
            for warning in diagnóstico["warnings"][:3]:  # Mostrar solo las primeras 3
                print(f"- {warning}")
        
        if diagnóstico["recommendations"]:
            print("\nRecomendaciones:")
            for rec in diagnóstico["recommendations"][:3]:  # Mostrar solo las primeras 3
                print(f"- {rec}")
        
        print(f"\nProceso completado. Resultados guardados en {results_dir}")
    else:
        print(f"\nERROR: No se encontraron resultados para el nodo de diálogo")

if __name__ == "__main__":
    print("\n¿QUÉ EJEMPLO DESEAS EJECUTAR?")
    print("1. Solución al problema de captura de diálogo (usando DialogWorkflowAPI)")
    print("2. Creación manual de workflow (mayor control)")
    
    option = input("\nSelecciona una opción (1/2): ")
    
    if option == "2":
        run_manual_workflow_example()
    else:
        run_enhanced_dialog_example()


¿QUÉ EJEMPLO DESEAS EJECUTAR?
1. Solución al problema de captura de diálogo (usando DialogWorkflowAPI)
2. Creación manual de workflow (mayor control)



Selecciona una opción (1/2):  2



EJEMPLO DE CREACIÓN MANUAL DE WORKFLOW - MAYOR CONTROL



Introduce un dominio para el diálogo manual (por ejemplo, 'inversiones', 'salud', etc.):  inversion
Introduce un prompt inicial para el diálogo:  estrategias de market neutral


[WorkflowMonitor.manual_workflow_inversion] Iniciando ejecución del workflow: manual_workflow_inversion
[10:37:23] INFO - WorkflowMonitor.manual_workflow_inversion: Iniciando ejecución del workflow: manual_workflow_inversion
[EnhancedWorkflowEngine.manual_workflow_inversion] Entrada proporcionada para nodo: expert_analyst_dialog
[10:37:23] INFO - EnhancedWorkflowEngine.manual_workflow_inversion: Entrada proporcionada para nodo: expert_analyst_dialog
[WorkflowMonitor.manual_workflow_inversion] Ejecución finalizada. Duración: 0.00 segundos
[10:37:23] INFO - WorkflowMonitor.manual_workflow_inversion: Ejecución finalizada. Duración: 0.00 segundos
[EnhancedWorkflowEngine.manual_workflow_inversion] Resultados guardados en: resultados_workflow_manual/manual_workflow_inversion_success_20250417_103723
[10:37:23] INFO - EnhancedWorkflowEngine.manual_workflow_inversion: Resultados guardados en: resultados_workflow_manual/manual_workflow_inversion_success_20250417_103723
[WorkflowMonitor.manual_wo


Ejecutando workflow manual con diálogo sobre inversion...
[manual_workflow_inversion] Iniciando ejecución del workflow: manual_workflow_inversion
[WorkflowEngine.manual_workflow_inversion] Entrada proporcionada para nodo: expert_analyst_dialog
[manual_workflow_inversion] Ejecución finalizada. Duración: 0.00 segundos
[WorkflowEngine.manual_workflow_inversion] Resultados guardados en: resultados_workflow_manual/manual_workflow_inversion_success_20250417_103723
[manual_workflow_inversion] Informe de ejecución guardado en: resultados_workflow_manual/manual_workflow_inversion_execution_report_20250417_103723.json

===== PUERTOS DE SALIDA DISPONIBLES =====
- default

===== DIAGNÓSTICO DEL WORKFLOW =====

Proceso completado. Resultados guardados en resultados_workflow_manual


In [3]:
import os
import sys
import logging
from typing import List, Dict, Any, Optional, Tuple
from datetime import datetime
from dotenv import load_dotenv

# Asegurarse de que puede encontrar el módulo incubator
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("../"))))

# Importar componentes de incubator
from incubator.llm.antropic import AnthropicClient
from incubator.agents.agent import Agent
from incubator.wf.enhanced_dialognode import EnhancedDialogNode
from incubator.wf.robust_workflow import RobustWorkflowEngine, create_robust_workflow
from incubator.debug.tracer import WorkflowTracer
from incubator.diagnostics.workflow_diagnostics import WorkflowDiagnostics

# Cargar variables de entorno
load_dotenv()

# Configurar logging
logging.basicConfig(
    level=logging.INFO,
    format='[%(asctime)s] %(levelname)s - %(name)s: %(message)s',
    datefmt='%H:%M:%S'
)

def run_robust_workflow_example(domain: str = None, prompt: str = None, debug_mode: bool = True):
    """
    Ejecuta un ejemplo de workflow robusto con diagnóstico y tracing avanzados
    
    Args:
        domain: Dominio para el diálogo
        prompt: Prompt inicial para el diálogo
        debug_mode: Si se debe activar modo de depuración
    """
    print("\n==================================================")
    print("EJEMPLO DE WORKFLOW ROBUSTO CON DIAGNÓSTICO AVANZADO")
    print("==================================================\n")
    
    # Solicitar dominio y prompt si no se proporcionaron
    if not domain:
        domain = input("Introduce un dominio para el diálogo (por ejemplo, 'inversiones', 'marketing', etc.): ")
        if not domain:
            domain = "estrategias de inversión"
    
    if not prompt:
        prompt = input("Introduce un prompt inicial para el diálogo: ")
        if not prompt:
            prompt = f"Desarrolla una estrategia de {domain} innovadora para un mercado volátil"
    
    # Obtener API key
    api_key = os.getenv("ANTHROPIC_API_KEY")
    if not api_key:
        print("ERROR: No se encontró ANTHROPIC_API_KEY en las variables de entorno")
        return
    
    # Cliente LLM
    llm_client = AnthropicClient(api_key=api_key)
    
    # Crear directorios para resultados
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    results_dir = f"resultados_robustos_{timestamp}"
    os.makedirs(results_dir, exist_ok=True)
    
    # Subdirectorios para diferentes tipos de salida
    traces_dir = os.path.join(results_dir, "traces")
    diagnostics_dir = os.path.join(results_dir, "diagnostics")
    outputs_dir = os.path.join(results_dir, "outputs")
    
    os.makedirs(traces_dir, exist_ok=True)
    os.makedirs(diagnostics_dir, exist_ok=True)
    os.makedirs(outputs_dir, exist_ok=True)
    
    try:
        # Crear los agentes
        expert = Agent(
            name="Expert",
            description=f"Experto en {domain}",
            llm_client=llm_client,
            system_prompt=f"""
            Eres un experto reconocido en el campo de {domain}.
            
            Tu tarea es proporcionar análisis profundos y perspectivas basadas en evidencia
            sobre temas relacionados con {domain}. Debes:
            
            1. Ofrecer explicaciones claras y detalladas
            2. Citar tendencias actuales y factores relevantes
            3. Proporcionar ejemplos concretos cuando sea necesario
            4. Responder a las preguntas del Analista de manera completa
            
            Mantén un tono profesional y asegúrate de que tus recomendaciones estén bien fundamentadas.
            """
        )
        
        analyst = Agent(
            name="Analyst",
            description=f"Analista crítico de {domain}",
            llm_client=llm_client,
            system_prompt=f"""
            Eres un analista crítico especializado en evaluar propuestas y análisis de {domain}.
            
            Tu tarea es examinar críticamente la información proporcionada por el Experto,
            identificar posibles sesgos, lagunas o áreas de mejora. Debes:
            
            1. Hacer preguntas incisivas que exploren diferentes ángulos
            2. Señalar posibles debilidades o limitaciones en los argumentos
            3. Sugerir consideraciones adicionales relevantes
            4. En tu evaluación final, proporcionar un análisis FODA (Fortalezas, Oportunidades, 
               Debilidades, Amenazas) de las ideas presentadas
            
            Cuando hayas completado tu análisis, marca claramente tu conclusión con "CONCLUSIÓN ANALÍTICA:"
            seguida de un resumen estructurado de las principales conclusiones.
            """
        )
        
        # Configurar marcadores de contenido
        content_markers = {
            "conclusion": "CONCLUSIÓN ANALÍTICA:",
            "swot_analysis": "ANÁLISIS FODA:",
            "recommendations": "RECOMENDACIONES:"
        }
        
        # Crear nodo de diálogo mejorado
        dialog_node = EnhancedDialogNode(
            id="expert_analyst_dialog",
            agent_a=expert,
            agent_b=analyst,
            max_iterations=3,
            terminate_on_keywords=list(content_markers.values()),
            content_markers=content_markers,
            expected_ports=["default", "conversation", "conclusion", "swot_analysis", 
                            "recommendations", "expert_final", "analyst_final"],
            enable_auto_correction=True,
            debug_mode=debug_mode,
            verbose=debug_mode
        )
        
        # Crear workflow robusto
        workflow = RobustWorkflowEngine(
            workflow_id=f"robust_{domain.replace(' ', '_')}",
            enable_diagnostics=True,
            enable_tracing=True,
            enable_validation=True,
            enable_auto_recovery=True,
            trace_output_dir=traces_dir,
            debug_mode=debug_mode,
            expected_node_outputs={
                "expert_analyst_dialog": ["default", "conversation", "conclusion", 
                                         "swot_analysis", "recommendations", 
                                         "expert_final", "analyst_final"]
            }
        )
        
        # Añadir nodo al workflow
        workflow.add_node(dialog_node, is_input=True, is_output=True)
        
        # Validar workflow antes de ejecutar
        print("\nValidando workflow...")
        validation_report = workflow.get_validation_report()
        
        if validation_report["is_valid"]:
            print("✅ Workflow validado correctamente")
        else:
            print(f"⚠️ Se encontraron {validation_report['error_count']} errores y {validation_report['warning_count']} advertencias")
            
            if validation_report["errors"]:
                print("\nErrores detectados:")
                for i, error in enumerate(validation_report["errors"], 1):
                    print(f"  {i}. {error}")
            
            if validation_report["warnings"]:
                print("\nAdvertencias:")
                for i, warning in enumerate(validation_report["warnings"], 1):
                    print(f"  {i}. {warning}")
            
            # Preguntar si se desea continuar
            if input("\n¿Desea continuar a pesar de los problemas? (s/n): ").lower() != 's':
                print("Ejecución cancelada por el usuario")
                return
        
        # Ejecutar diagnóstico inicial
        print("\nRealizando diagnóstico inicial...")
        initial_diagnosis = workflow.run_diagnostics()
        
        if initial_diagnosis["status"] == "healthy":
            print("✅ Diagnóstico inicial: Workflow en buen estado")
        else:
            print(f"⚠️ Diagnóstico inicial: {len(initial_diagnosis['issues'])} problemas, {len(initial_diagnosis['recommendations'])} recomendaciones")
            
            if initial_diagnosis["recommendations"]:
                print("\nRecomendaciones:")
                for i, rec in enumerate(initial_diagnosis["recommendations"], 1):
                    print(f"  {i}. {rec}")
        
        # Ejecutar workflow
        print(f"\nEjecutando workflow robusto para {domain}...")
        print(f"Prompt: '{prompt}'")
        print("\nEsto puede tomar un tiempo. Por favor, espere...\n")
        
        # Ejecutar con todas las capacidades activadas
        results = workflow.execute(
            inputs={"expert_analyst_dialog": prompt},
            save_results=True,
            results_dir=outputs_dir
        )
        
        # Mostrar resultados disponibles
        if "expert_analyst_dialog" in results:
            outputs = results["expert_analyst_dialog"]
            
            print("\n===== PUERTOS DE SALIDA DISPONIBLES =====")
            for port in outputs.keys():
                print(f"- {port}")
            
            # Mostrar análisis FODA si está disponible
            if "swot_analysis" in outputs:
                print("\n===== ANÁLISIS FODA =====")
                swot = outputs["swot_analysis"]["content"]
                # Limitar a primeras 300 caracteres para la salida
                preview = swot[:300] + "..." if len(swot) > 300 else swot
                print(preview)
            
            # Mostrar conclusión si está disponible
            if "conclusion" in outputs:
                print("\n===== CONCLUSIÓN ANALÍTICA =====")
                conclusion = outputs["conclusion"]["content"]
                # Limitar a primeras 300 caracteres para la salida
                preview = conclusion[:300] + "..." if len(conclusion) > 300 else conclusion
                print(preview)
            
            # Guardar informe de diagnóstico final
            workflow.save_diagnostic_report(diagnostics_dir)
            
            # Mostrar estadísticas de tracing
            if workflow.enable_tracing:
                trace_summary = workflow.get_trace_summary()
                
                print("\n===== ESTADÍSTICAS DE EJECUCIÓN =====")
                print(f"Tiempo total: {trace_summary.get('execution_time', 'N/A'):.2f}s")
                print(f"Nodos ejecutados: {trace_summary.get('nodes_executed', 'N/A')}")
                print(f"Total de eventos: {trace_summary.get('total_events', 'N/A')}")
                print(f"Errores: {trace_summary.get('error_count', 'N/A')}")
            
            print(f"\nProceso completado. Resultados guardados en {results_dir}")
        else:
            print(f"\nERROR: No se encontraron resultados para el nodo de diálogo")
    
    except Exception as e:
        import traceback
        print(f"\nERROR durante la ejecución: {str(e)}")
        print(traceback.format_exc())
        print(f"\nConsulte los archivos de diagnóstico en {diagnostics_dir} para más detalles")


def create_problematic_workflow():
    """
    Crea un workflow con problemas intencionados para probar el sistema de diagnóstico
    """
    print("\n==================================================")
    print("EJEMPLO DE DIAGNÓSTICO DE WORKFLOW PROBLEMÁTICO")
    print("==================================================\n")
    
    # Obtener API key
    api_key = os.getenv("ANTHROPIC_API_KEY")
    if not api_key:
        print("ERROR: No se encontró ANTHROPIC_API_KEY en las variables de entorno")
        return
    
    # Cliente LLM
    llm_client = AnthropicClient(api_key=api_key)
    
    # Crear directorios para resultados
    results_dir = "resultados_diagnostico"
    os.makedirs(results_dir, exist_ok=True)
    
    # Crear agentes pero intencionalmente configurarlos mal
    expert = Agent(
        name="Expert",
        description="Experto en inversiones",
        llm_client=llm_client,
        system_prompt="Eres un experto en inversiones."
    )
    
    # No proporcionar agent_b correctamente
    analyst = None
    
    # Crear nodo de diálogo con problemas
    dialog_node = EnhancedDialogNode(
        id="problematic_dialog",
        agent_a=expert,
        agent_b=analyst,  # Esto causará problemas
        max_iterations=-1,  # Valor inválido
        content_markers={},  # Sin marcadores
        expected_ports=["default", "conversation"],
        debug_mode=True,
        verbose=True
    )
    
    # Crear workflow robusto para diagnóstico
    workflow = RobustWorkflowEngine(
        workflow_id="problematic_workflow",
        enable_diagnostics=True,
        enable_validation=True,
        enable_auto_recovery=True,
        debug_mode=True
    )
    
    # Añadir nodo al workflow
    workflow.add_node(dialog_node, is_input=True, is_output=True)
    
    # Ejecutar diagnóstico
    print("\nEjecutando diagnóstico del workflow problemático...")
    diagnosis = workflow.run_diagnostics()
    
    # Mostrar resultados
    print(f"\nEstado del workflow: {diagnosis['status']}")
    print(f"Se encontraron {len(diagnosis['issues'])} problemas")
    
    if diagnosis["issues"]:
        print("\nProblemas detectados:")
        for i, issue in enumerate(diagnosis["issues"], 1):
            print(f"{i}. {issue}")
    
    if diagnosis["recommendations"]:
        print("\nRecomendaciones:")
        for i, rec in enumerate(diagnosis["recommendations"], 1):
            print(f"{i}. {rec}")
    
    # Intentar reparación automática
    print("\nIntentando reparación automática...")
    repair_results = workflow.diagnostics.attempt_repair()
    
    if repair_results["workflow_modified"]:
        print("\nReparaciones aplicadas:")
        for repair in repair_results["repairs_successful"]:
            print(f"✅ {repair}")
        
        for repair in repair_results["repairs_failed"]:
            print(f"❌ {repair}")
        
        # Ejecutar diagnóstico después de reparación
        print("\nEjecutando diagnóstico después de reparación...")
        post_repair_diagnosis = workflow.run_diagnostics()
        
        print(f"\nEstado después de reparación: {post_repair_diagnosis['status']}")
        print(f"Problemas restantes: {len(post_repair_diagnosis['issues'])}")
    else:
        print("No se pudieron aplicar reparaciones automáticas")
    
    # Guardar informe de diagnóstico
    diagnosis_file = workflow.save_diagnostic_report(results_dir)
    print(f"\nInforme de diagnóstico guardado en: {diagnosis_file}")


if __name__ == "__main__":
    print("\n¿QUÉ EJEMPLO DESEAS EJECUTAR?")
    print("1. Workflow robusto con diagnóstico y tracing")
    print("2. Diagnóstico de workflow problemático")
    
    option = input("\nSelecciona una opción (1/2): ")
    
    if option == "2":
        create_problematic_workflow()
    else:
        run_robust_workflow_example()


¿QUÉ EJEMPLO DESEAS EJECUTAR?
1. Workflow robusto con diagnóstico y tracing
2. Diagnóstico de workflow problemático



Selecciona una opción (1/2):  1



EJEMPLO DE WORKFLOW ROBUSTO CON DIAGNÓSTICO AVANZADO



Introduce un dominio para el diálogo (por ejemplo, 'inversiones', 'marketing', etc.):  inversion
Introduce un prompt inicial para el diálogo:  


[WorkflowDiagnostics] Iniciando diagnóstico completo del workflow
[10:47:22] INFO - WorkflowDiagnostics: Iniciando diagnóstico completo del workflow
[WorkflowDiagnostics] Iniciando diagnóstico completo del workflow
[10:47:22] INFO - WorkflowDiagnostics: Iniciando diagnóstico completo del workflow
[10:47:22] [WorkflowTracer.robust_inversion] Iniciando tracing para workflow: robust_inversion
[10:47:22] INFO - WorkflowTracer.robust_inversion: Iniciando tracing para workflow: robust_inversion
[10:47:22] [WorkflowTracer.robust_inversion] • Evento execute_start: {'inputs': ['expert_analyst_dialog'], 'has_context
[10:47:22] INFO - WorkflowTracer.robust_inversion: • Evento execute_start: {'inputs': ['expert_analyst_dialog'], 'has_context
[WorkflowMonitor.robust_inversion] Iniciando ejecución del workflow: robust_inversion
[10:47:22] INFO - WorkflowMonitor.robust_inversion: Iniciando ejecución del workflow: robust_inversion
[EnhancedWorkflowEngine.robust_inversion] Entrada proporcionada para no


Validando workflow...
✅ Workflow validado correctamente

Realizando diagnóstico inicial...
[Diagnostics] Iniciando diagnóstico completo del workflow
⚠️ Diagnóstico inicial: 1 problemas, 6 recomendaciones

Recomendaciones:
  1. Verificar la estructura del grafo para asegurar que no hay ciclos ni nodos aislados
  2. Asegurar que todos los nodos de entrada pueden alcanzar al menos un nodo de salida
  3. Verificar la configuración de los nodos de diálogo, especialmente los agentes y marcadores de contenido
  4. Considerar usar el modo de depuración en los nodos de diálogo para obtener más información
  5. Considerar agregar validación previa a la ejecución del workflow
  6. Implementar manejo de errores más detallado para diagnosticar problemas específicos

Ejecutando workflow robusto para inversion...
Prompt: 'Desarrolla una estrategia de inversion innovadora para un mercado volátil'

Esto puede tomar un tiempo. Por favor, espere...

[Diagnostics] Iniciando diagnóstico completo del workf